In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import time

In [2]:
# Define paramaters for the model
learning_rate = 0.01
batch_size = 128
n_epochs = 25

Step 1: Read in data<br>
using TF Learn's built in function to load MNIST data to the folder data/mnist

In [3]:
mnist = input_data.read_data_sets('../data/mnist', one_hot=True) 

Extracting ../data/mnist/train-images-idx3-ubyte.gz
Extracting ../data/mnist/train-labels-idx1-ubyte.gz
Extracting ../data/mnist/t10k-images-idx3-ubyte.gz
Extracting ../data/mnist/t10k-labels-idx1-ubyte.gz


In [4]:
#print(mnist.train.labels)
print(mnist.test.num_examples)

10000


Step 2: create placeholders for features and labels<br>
each image in the MNIST data is of shape 28*28 = 784<br>
therefore, each image is represented with a 1x784 tensor<br>
there are 10 classes for each image, corresponding to digits 0 - 9.<br>
Features are of the type float, and labels are of the type int<br>

In [5]:
sz_img = 784
num_labels = 10
X = tf.placeholder(dtype=tf.float32, shape=(None, sz_img), name='data')
Y = tf.placeholder(dtype=tf.int16, shape=(None, num_labels), name='labels')

Step 3: create weights and bias<br>
weights and biases are initialized to 0<br>
shape of w depends on the dimension of X and Y so that Y = X * w + b<br>
shape of b depends on Y<br>

In [6]:
w = tf.Variable(initial_value=tf.zeros(shape=(sz_img, num_labels)), name='w', dtype=tf.float32)
b = tf.Variable(initial_value=0, expected_shape=Y.shape, name='b', dtype=tf.float32)

Step 4: build model<br>
the model that returns the logits.<br>
this logits will be later passed through softmax layer<br>
to get the probability distribution of possible label of the image<br>
DO NOT DO SOFTMAX HERE<br>

In [7]:
# Got caught with 'X * w + b' with such error: https://stackoverflow.com/questions/42407628/valueerror-dimensions-must-be-equal-but-are-784-and-500-for-mul-op-mul
logits = tf.matmul(X, w) + b

In [8]:
print(w.shape)
print(X.shape)
print(Y.shape)
print(logits.shape)

(784, 10)
(?, 784)
(?, 10)
(?, 10)


Step 5: define loss function<br>
use cross entropy loss of the real labels with the softmax of logits<br>
use the method:<br>
tf.nn.softmax_cross_entropy_with_logits(logits, Y)<br>
then use tf.reduce_mean to get the mean loss of the batch<br>

In [9]:
# we reduce the vector of size `num_train_elems`, that denotes "-Sigma(o_i * log(predicted_i))" for each training sample
raw_losses = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y) 
loss = tf.reduce_mean(raw_losses, axis=0, name='loss')
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(loss)

Step 6: define training op<br>
using gradient descent to minimize loss

In [10]:
with tf.Session() as sess:
	start_time = time.time()
	sess.run(tf.global_variables_initializer())	
	n_batches = int(mnist.train.num_examples/batch_size)
	for i in range(n_epochs): # train the model n_epochs times
		total_loss = 0

		for _ in range(n_batches):
			X_batch, Y_batch = mnist.train.next_batch(batch_size)
			_, loss_batch = sess.run([optimizer, loss], feed_dict={X: X_batch, Y: Y_batch})
			total_loss += loss_batch
		print('Average loss epoch {0}: {1}'.format(i, total_loss/n_batches))

	print('Total time: {0} seconds'.format(time.time() - start_time))

	print('Optimization Finished!') # should be around 0.35 after 25 epochs

	# test the model
	preds = tf.nn.softmax(logits)
	correct_preds = tf.equal(tf.argmax(preds, 1), tf.argmax(Y, 1))
	accuracy = tf.reduce_sum(tf.cast(correct_preds, tf.float32), axis=0, name='accuracy')
	
    # Dividing test on batches is incorrect, because there are 10_000 tests and batch_size is 128.
    # It does not divide evenly, so the last batch (which is not full) is omitted.
    # So I calculate accuracy on full testset at once.
	total_correct_preds = sess.run(accuracy, feed_dict={X: mnist.test.images, Y: mnist.test.labels})

	print('Accuracy {0}'.format(total_correct_preds/mnist.test.num_examples))

Average loss epoch 0: 1.2897358389445397
Average loss epoch 1: 0.7343404270830132
Average loss epoch 2: 0.6023448370136582
Average loss epoch 3: 0.5387209714292647
Average loss epoch 4: 0.500982777430461
Average loss epoch 5: 0.473141751783989
Average loss epoch 6: 0.45481212850495095
Average loss epoch 7: 0.43811853268207646
Average loss epoch 8: 0.4258927981128226
Average loss epoch 9: 0.4179660342025868
Average loss epoch 10: 0.4075173890535092
Average loss epoch 11: 0.4002338411646845
Average loss epoch 12: 0.3944635681616954
Average loss epoch 13: 0.3885146698334834
Average loss epoch 14: 0.3822426192966097
Average loss epoch 15: 0.3795846157418542
Average loss epoch 16: 0.37431621801603093
Average loss epoch 17: 0.37107486223979985
Average loss epoch 18: 0.36556209856535726
Average loss epoch 19: 0.36572009189562366
Average loss epoch 20: 0.36056160704517143
Average loss epoch 21: 0.3580765082980647
Average loss epoch 22: 0.3556067767815712
Average loss epoch 23: 0.35253546169865